## 🚀 Unlock the Power of Groq: A Hands-on API Exploration!
> This notebook is your launchpad into the exciting world of Groq.

__Get ready to dive into:__

🪐 Hello Groq: Take your first steps with the Groq client and witness its capabilities firsthand.

⚡ Streaming with Groq: Harness the power of real-time data processing with Groq's efficient streaming functionalities.

🛠️ JSON Mode Unleashed: Discover the flexibility of working with JSON Data Models.

🧰 Tools: Explore how we cal use functional calling with custom Tools.

P.S. Don't forget to grab your API key from the [Groq Console](https://console.groq.com/keys) before embarking on this journey!

Authored by: [Hemanth HM](https://h3manth.com)

### Install deps

In [1]:
!pip install -qU groq

### Export API KEY

In [9]:
%env GROQ_API_KEY=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

env: GROQ_API_KEY=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


## Hello Groq

In [10]:
from groq import Groq

client = Groq()

user_content = "Give me 5 dadjokes" #input("groq> ")

chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": user_content}
    ],
    model="llama3-70b-8192",
    stream=True,

)

for chunk in chat_completion:
    print(chunk.choices[0].delta.content, end="")

Here are five dadjokes for you:

1. Why did the scarecrow win an award? Because he was outstanding in his field! (get it?)
2. I told my wife she was drawing her eyebrows too high. She looked surprised.
3. Why do chicken coops only have two doors? Because if they had four, they would be a sedan.
4. What do you call a fake noodle? An impasta.
5. Why did the mushroom go to the party? Because he was a fun-gi.

I hope you groaned at least once!None

# Streaming 

In [18]:
!pip install -qU asyncio nest_asyncio

In [23]:
import asyncio
import nest_asyncio
from groq import AsyncGroq

nest_asyncio.apply() # need for notebooks

async def main():
    client = AsyncGroq()

    stream = await client.chat.completions.create(
        messages=[
            {"role": "system", "content": "You an expert comedian, with two decades of experience"},
            {"role": "user", "content": "Give me 5 dadjokes"},
        ],
        model="llama3-70b-8192",
        temperature=0.5,
        max_tokens=1024,
        top_p=1,
        stream=True,
    )

    async for chunk in stream:
        print(chunk.choices[0].delta.content, end="")

import asyncio
loop = asyncio.get_event_loop()
loop.create_task(main())

<Task pending name='Task-142' coro=<main() running at /tmp/ipykernel_1938/2813795872.py:7>>

You want some groan-inducing, eye-rolling-worthy, and utterly fantastic dad jokes, eh? Alright, here are five of 'em:

1. Why did the scarecrow win an award? Because he was outstanding in his field! (get it?)
2. I told my wife she was drawing her eyebrows too high. She looked surprised.
3. Why do chicken coops only have two doors? Because if they had four, they would be a sedan!
4. What do you call a fake noodle? An impasta!
5. Why did the mushroom go to the party? Because he was a fun-gi!

Remember, these jokes are so bad, they're good!None

### JSON Mode (non-streaming)

In [19]:
from typing import List, Optional
import json
from pydantic import BaseModel

In [20]:
# Data models (from groq example)
class Ingredient(BaseModel):
    name: str
    quantity: str
    quantity_unit: Optional[str]

class Recipe(BaseModel):
    recipe_name: str
    ingredients: List[Ingredient]
    directions: List[str]

In [12]:
# Function to create the system message for Groq
def create_system_message(schema: str) -> dict:
    return {
        "role": "system",
        "content": "You are master chef with a great source of repeices, outputs recipes in JSON.\n"
        f" The JSON object must use the schema: {schema}",
    }

In [13]:
# Function to create the user message for Groq
def create_user_message(recipe_name: str) -> dict:
    return {"role": "user", "content": f"Fetch a recipe for {recipe_name}"}

In [14]:
# Function to fetch recipe data from Groq
def fetch_recipe_data(recipe_name: str, groq_client: Groq) -> dict:
    messages = [
        create_system_message(json.dumps(Recipe.model_json_schema(), indent=2)),
        create_user_message(recipe_name),
    ]
    response = groq_client.chat.completions.create(
        messages=messages,
        model="mixtral-8x7b-32768",
        temperature=0,
        stream=False,
        response_format={"type": "json_object"},
    )
    return response.choices[0].message.content

In [15]:
# Function to parse recipe data into a Recipe object
def parse_recipe(recipe_data: dict) -> Recipe:
    return Recipe.model_validate_json(recipe_data)

In [16]:
# Function to format an ingredient string
def format_ingredient(ingredient: Ingredient) -> str:
    return f"- {ingredient.name}: {ingredient.quantity} {ingredient.quantity_unit or ''}"

In [17]:
# Function to format a direction string
def format_direction(step: int, direction: str) -> str:
    return f"{step}. {direction}"

In [18]:
# Function to print the recipe
def print_recipe(recipe: Recipe):
    print("Recipe:", recipe.recipe_name)
    print("\nIngredients:")
    print("\n".join(map(format_ingredient, recipe.ingredients)))
    print("\nDirections:")
    print("\n".join(map(format_direction, range(1, len(recipe.directions) + 1), recipe.directions)))

In [19]:
# Main function (composition of functions)
def cook(recipe_name: str, groq_client: Groq):
    recipe_data = fetch_recipe_data(recipe_name, groq_client)
    recipe = parse_recipe(recipe_data)
    print_recipe(recipe)

In [21]:
cook('Bise Bele Bath', client) # https://en.wikipedia.org/wiki/Bisi_Bele_Bath

Recipe: Bise Bele Bath

Ingredients:
- Toor Dal: 1 cup
- Chana Dal: 1/4 cup
- Moong Dal: 1/4 cup
- Tamarind: 1 lemon-sized ball
- Jaggery: 1 inch cube
- Turmeric Powder: 1/2 tsp
- Sambar Powder: 2 tsp
- Rasam Powder: 1 tsp
- Salt: to taste 
- Oil: 2 tsp
- Mustard Seeds: 1/2 tsp
- Curry Leaves: 10-12 
- Dry Red Chillies: 2-3 
- Asafoetida: a pinch 
- Onion: 1 medium-sized
- Garlic: 3-4 cloves
- Ginger: 1 inch
- Green Chillies: 2-3 
- Coriander Leaves: 2-3 tbsp

Directions:
1. Soak tamarind in 2 cups of warm water for 15-20 minutes.
2. Extract the juice and discard the pulp.
3. Wash and soak all the dals together for 30 minutes.
4. Pressure cook the dals along with tamarind juice, jaggery, turmeric powder, sambar powder, rasam powder, and salt for 3-4 whistles or until the dal is soft and mushy.
5. Heat oil in a pan, add mustard seeds, curry leaves, dry red chillies, and asafoetida.
6. Once the mustard seeds splutter, add chopped onion, minced garlic, and grated ginger.
7. Saute until th

### Tools

In [21]:
import json
from groq import Groq

MODEL = 'mixtral-8x7b-32768'

In [14]:
def convert(value: float, in_metric: str, out_metric: str) -> float:
    """Converts a value between units. 
    Currently only supports miles to kilometers.
    
    Args:
        value: The numerical value to convert.
        in_metric: The input unit (e.g., "mile").
        out_metric: The desired output unit (e.g., "km").

    Returns:
        The converted value.
    
    Raises:
        ValueError: If the conversion is not supported.
    """
    print(value, in_metric, out_metric)
    if in_metric.lower() == "mile" and out_metric.lower() == "km":
        return value * 1.60934
    else:
        raise ValueError("Unsupported conversion: I only do miles to kms for now :D")


In [3]:
def construct_messages(user_prompt: str) -> list[dict]:
    """Constructs the initial list of messages for the conversation.
    
    Args:
        user_prompt: The user's input prompt.

    Returns:
        A list of dictionaries representing the messages.
    """

    return [
        {"role": "system", "content": "You are a function calling LLM..."},
        {"role": "user", "content": user_prompt},
    ]

In [4]:
def construct_tools() -> list[dict]:
    """Defines the available tools (functions) for the LLM.

    Returns:
        A list of dictionaries describing the tools.
    """
    return [
        {
            "type": "function",
            "function": {
                "name": "convert",
                "description": "Convert between units",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "value": {
                            "type": "number",
                            "description": "the value that we are trying to convert",
                        },
                        "in_metric": {
                            "type": "string",
                            "description": "the input metric"
                        },
                        "out_metric": {
                            "type": "string",
                            "description": "the output metric"
                        }
                    },
                    "required": ["value", "in_metric", "out_metric"],
                },
            },
        }
    ]

In [12]:
def call_function(tool_call: dict, available_functions: dict) -> dict:
    """Calls the specified function with the provided arguments.
    
    Args:
        tool_call: A dictionary containing information about the function call.
        available_functions: A dictionary mapping function names to functions.

    Returns:
        A dictionary representing the function call response.
    """
    function_name = tool_call.function.name  # Access attributes using dot notation
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args) 
    return {
        "tool_call_id": tool_call.id,  # Access 'id' attribute directly
        "role": "tool",
        "name": function_name,
        "content": str(function_response),
    }


In [6]:
def process_tool_calls(
    messages: list[dict], tool_calls: list[dict], available_functions: dict
) -> list[dict]:
    """Processes tool calls by calling the functions and adding responses to messages.

    Args:
        messages: The current list of messages in the conversation.
        tool_calls: A list of tool calls made by the LLM.
        available_functions: A dictionary mapping function names to functions.

    Returns:
        An updated list of messages including tool call responses.
    """

    return messages + [
        call_function(tool_call, available_functions) for tool_call in tool_calls
    ]


In [7]:
def run_conversation(client: Groq, user_prompt: str) -> str:
    """Runs a conversation with the LLM, handling function calls and responses.

    Args:
        client: The Groq client instance. 
        user_prompt: The user's input prompt.

    Returns:
        The final response from the LLM.
    """

    messages = construct_messages(user_prompt)
    tools = construct_tools()

    response = client.chat.completions.create(
        model=MODEL, messages=messages, tools=tools, tool_choice="auto", max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    if tool_calls:
        available_functions = {"convert": convert}
        messages = process_tool_calls(
            messages + [response_message], tool_calls, available_functions
        )
        second_response = client.chat.completions.create(model=MODEL, messages=messages)
        return second_response.choices[0].message.content 

In [18]:
user_prompt = "Convert 1 mile to km"
result = run_conversation(client, user_prompt)
print(result)


1 mile km
The conversion of 1 mile to kilometers is approximately 1.60934. This result reflects the distance in kilometers, which is the metric unit equivalent of 1 mile.
